## On essaye de faire un truc qui marche ajd

In [13]:
from Bio import Entrez, SeqIO
Entrez.email = "Chevreau.julien21@gmail.com@univ-rouen.fr"

input = ""

db="protein"
term = "homo_sapiens[Orgn] AND RAD51[gene]"

handle = Entrez.esearch(db=db, term=term, retmax=100)
records = Entrez.read(handle)

identifiers = records["IdList"]
print(identifiers)
handle = Entrez.efetch(db=db, id=identifiers, retmax="100",
rettype="gbk", retmode="text")
text = handle.read()


with open("RAD51_prot.gbk","w") as gbk:
    gbk.write(text)

for seq_record in SeqIO.parse("RAD51.gbk", "genbank"):
        print("Name %s, %ibp, annotation : %s"
            % (seq_record.name, len(seq_record), seq_record.annotations))


['548663', '256017145', '256017143', '256017141', '19924133', '2462545405', '2462545403', '2462545401', '2462545399', '2462545397', '2462545395', '2217302260', '2217302258', '2217302253', '767984894', '767984892', '767984890', '47680391', '119612841', '119612840', '119612839', '119612838', '119612837', '12655203', '27368250', '49168602', '285977']
Name NM_001164269, 2335bp, annotation : {'molecule_type': 'mRNA', 'topology': 'linear', 'data_file_division': 'PRI', 'date': '17-NOV-2024', 'accessions': ['NM_001164269'], 'sequence_version': 2, 'keywords': ['RefSeq'], 'source': 'Homo sapiens (human)', 'organism': 'Homo sapiens', 'taxonomy': ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo'], 'references': [Reference(title='Mechanism of BCDX2-mediated RAD51 nucleation on short ssDNA stretches and fork DNA', ...), Reference(title='Hyper-recombination in ribosoma

## Prot et transcrits traduits 
Aller chercher dans la base **prot** pour avoir les différents IDs des prots et transcrits associés

In [25]:
for seq_record in SeqIO.parse("RAD51_prot.gbk", "genbank"):
    if seq_record.annotations["db_source"][:6] == "REFSEQ" :
        print(seq_record.id)
        print(seq_record.annotations["db_source"].split(" ")[-1])

NP_001157742.1
NP_001157741.1
NP_597994.3
NP_002866.2
XP_054234546.1
XP_054234545.1
XP_054234544.1
XP_054234543.1
XP_054234542.1
XP_054234541.1
XP_047288881.1
XP_011520163.2
XP_011520159.2
XP_011520162.1
XP_011520161.1
XP_011520160.1


## Transcrits non traduits
Aller dans **nucleotide** pour les transcrits non traduits

In [ ]:
for seq_record in SeqIO.parse("RAD51.gbk", "genbank"):
    if seq_record.annotations["db_source"][:6] == "REFSEQ" :
        print(seq_record.id)
        print(seq_record.annotations["db_source"].split(" ")[-1])

## Gène
Aller dans **gene** pour l'official full name et le gene access number

In [25]:
from Bio import SeqIO
gb_file = "all_from_eu.gbk"
for seq_record in SeqIO.parse(gb_file, "genbank"):
    print("Name %s, %ibp"
          % (seq_record.name, len(seq_record)))
    #print(repr(seq_record.seq))


Name PQ523384, 252930bp
Name PQ165104, 289334bp
Name PP663685, 262916bp
Name PP663684, 263010bp
Name PP916044, 251922bp
Name PP579759, 219776bp
Name PP780168, 219775bp
Name NC_088489, 287626bp
Name OM867576, 239184bp
Name OM867577, 258440bp
Name PP556433, 287626bp
Name PP231953, 219775bp
Name NC_085821, 265457bp
Name NC_085700, 288122bp
Name NC_085699, 287054bp
Name NC_085616, 196604bp
Name PP035758, 219774bp
Name OQ916154, 265457bp
Name ON244455, 359596bp
Name ON244454, 219771bp
Name BK063798, 288122bp
Name BK063239, 287054bp
Name OQ701529, 196604bp
Name NC_081483, 334431bp
Name OQ852786, 334431bp
Name NC_016120, 232241bp
Name NC_042143, 271601bp
Name NC_049892, 219736bp
Name NC_042883, 287405bp
Name NC_037304, 367808bp
Name NC_037070, 323159bp
Name NC_031896, 240024bp
Name NC_029182, 232407bp
Name NC_018551, 258426bp
Name NC_016123, 219766bp
Name NC_016125, 219747bp
Name NC_016118, 360271bp
Name NC_008285, 221853bp
Name OP729396, 219779bp
Name ON758780, 175791bp
Name ON758779, 65849b

In [26]:
from Bio import SeqIO
SeqIO.convert("all_from_eu.gbk",'genbank',"all_from_eu.fasta","fasta")

93

In [53]:
from Bio import SeqIO
gb_file = "f1.gbk"
features = {}
genes = {}
for seq_record in SeqIO.parse(gb_file, "genbank"):
    for feat in seq_record.features :
        if feat.type in features :
            features[feat.type]+=1
        else : 
            features[feat.type] = 1
        if "gene" in feat.qualifiers:
            for gene in feat.qualifiers["gene"]:
                genes[gene] = 1
                
print(features)
print(genes)

{'source': 1, 'gene': 131, 'CDS': 117, 'regulatory': 14, 'rRNA': 3, 'exon': 32, 'misc_feature': 455, 'intron': 21, 'tRNA': 21, 'repeat_region': 6, 'STS': 1}
{'nad2': 1, 'rrn26': 1, 'nad5': 1, 'nad9': 1, 'rpl16': 1, 'rps3': 1, 'tRNA-Lys': 1, 'ccb206': 1, 'cox2': 1, 'ccb452': 1, 'tRNA-Gly': 1, 'rpl5': 1, 'cob': 1, 'tRNA-Ser': 1, 'tRNA-fMet': 1, 'nad6': 1, 'rps4': 1, 'tRNA-Tyr': 1, 'tRNA-Pro': 1, 'tRNA-Cys': 1, 'tRNA-Asn': 1, 'atp6-1': 1, 'tRNA-Ile': 1, 'orfB': 1, 'nad7': 1, 'nad1': 1, 'matR': 1, 'rpl2': 1, 'orfX': 1, 'nad4': 1, 'orf25': 1, 'nad4L': 1, 'cox3': 1, 'tRNA-Glu': 1, 'ccb382': 1, 'ccb256': 1, 'tRNA-Trp': 1, 'tRNA-Gln': 1, 'ccb203': 1, 'rps12': 1, 'tRNA-Asp': 1, 'atp9': 1, 'atp6-2': 1, 'atp1': 1, 'rps7': 1, 'tRNA-met': 1, 'cox1': 1, 'rrn5': 1, 'rrn18': 1}


## Info à récupérer  
- Gene symbol
- Official full name
- Gene access number 
- RNA access number